<a href="https://colab.research.google.com/github/bhupeshks21/projects/blob/main/blinkit_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @markdown BLinkit Analysis
# @markdown importing librarires

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# @title raw data
df=pd.read_csv("/content/blinkit_data.csv")

In [ ]:
# @title sample data
df.head(10)

In [ ]:
df.shape

In [ ]:
# @title data field
df.columns

In [ ]:
df.dtypes

In [ ]:
# @title cleaning
print(df['Item Fat Content'].unique())


In [ ]:
df['Item Fat Content']=df['Item Fat Content'].replace({'LF': 'Low Fat','low fat' : 'Low Fat' ,'reg':  'Regular'})

In [ ]:
# @title Business Requirements
# total sales
total_sales =df['Sales'].sum()
avg_sales =df['Sales'].mean()
#nof od items sales
no_of_items_sold=df['Sales'].count()
#Average_rating
avg_ratings=df['Rating'].mean()

print(f"Total Sales : ${total_sales:,.0f}")
print(f"avg_sales  : {avg_sales:,.0f}")
print(f"no_of_items_sold : {no_of_items_sold:,.0f}")
print(f"avg_rating : {avg_ratings:,.1f}")

In [ ]:
# @title Chart requirememt

sales_by_fat=df.groupby('Item Fat Content')['Sales'].sum()
plt.pie(sales_by_fat,labels=sales_by_fat.index,
        autopct = '%.1f%%',
        startangle=90)
plt.title('sales by fat content')
plt.axis('equal')
plt.show()

In [ ]:
sales_by_type = df.groupby('Item Type')['Sales'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
bars = plt.bar(sales_by_type.index, sales_by_type.values)

plt.xticks(rotation=-90)
plt.xlabel('Item Type')
plt.ylabel('Total Sales')
plt.title('Total Sales by Item Type')

for bar in bars:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
    f'{bar.get_height():.0f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
grouped = df.groupby(['Outlet Location Type', 'Item Fat Content'])['Sales'].sum().unstack()
grouped = grouped[['Regular', 'Low Fat']]
ax = grouped.plot(kind='bar', figsize=(8, 5), title='Outlet Tier by Item Fat Content')
plt.xlabel('Outlet Location Tier')
plt.ylabel('Total Sales')
plt.legend(title='Item Fat Content')
plt.tight_layout()
plt.show()

In [ ]:
sales_by_year = df.groupby('Outlet Establishment Year')['Sales'].sum().sort_index()

plt.figure(figsize=(9,5))
plt.plot(sales_by_year.index, sales_by_year.values, marker='o', linestyle='-')

plt.xlabel('Outlet Establishment Year')
plt.ylabel('Total Sales')
plt.title('Outlet Establishment')

for x, y in zip(sales_by_year.index, sales_by_year.values):
    plt.text(x, y, f'{y:,.0f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()